In [13]:
import plotly.graph_objects as go
from scores.categorical import firm
import xarray as xr
import numpy as np

In [14]:
fcst = xr.open_mfdataset(["../data/fcst_2020_2021.nc", "../data/fcst_2021_2022.nc", "../data/fcst_2022_2023.nc"])
fcst = fcst["__xarray_dataarray_variable__"].compute()

obs = xr.open_mfdataset(["../data/obs_2020_2021.nc", "../data/obs_2021_2022.nc", "../data/obs_2022_2023.nc"])
obs = obs["__xarray_dataarray_variable__"].compute()

In [15]:
risk_parameter = 0.5
categorical_thresholds = [1, 3]
threshold_weights = [2, 1]
firm_score = firm(fcst, obs, risk_parameter, categorical_thresholds, threshold_weights, preserve_dims="lead_day")
firm_score

<xarray.Dataset>
Dimensions:                (lead_day: 7)
Coordinates:
  * lead_day               (lead_day) int64 0 1 2 3 4 5 6
Data variables:
    firm_score             (lead_day) float64 0.00753 0.01514 ... 0.03663
    overforecast_penalty   (lead_day) float64 0.003929 0.009904 ... 0.02567
    underforecast_penalty  (lead_day) float64 0.003601 0.005238 ... 0.01096

In [16]:
# Calculate benchmark score of never warning
firm_ref = firm(fcst*0, obs, risk_parameter, categorical_thresholds, threshold_weights, preserve_dims="lead_day")
firm_ref

<xarray.Dataset>
Dimensions:                (lead_day: 7)
Coordinates:
  * lead_day               (lead_day) int64 0 1 2 3 4 5 6
Data variables:
    firm_score             (lead_day) float64 0.02638 0.02638 ... 0.02638
    overforecast_penalty   (lead_day) float64 0.0 0.0 0.0 0.0 0.0 0.0 0.0
    underforecast_penalty  (lead_day) float64 0.02638 0.02638 ... 0.02638

In [27]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=firm_score.lead_day, 
        y=firm_score.firm_score, 
        name='FIRM score', 
        line=dict(color="#0072B2")
        )
    )
fig.add_trace(
    go.Scatter(
        x=firm_score.lead_day, 
        y=firm_score.overforecast_penalty, 
        name='Overforecast penalty', 
        line=dict(color="#D55E00")
        )
    )
fig.add_trace(
    go.Scatter(
        x=firm_score.lead_day, 
        y=firm_score.underforecast_penalty, 
        name='Underforecast penalty', 
        line=dict(color="#CC79A7")
        )
    )
fig.add_trace(
    go.Scatter(
        x=firm_ref.lead_day, 
        y=firm_ref.firm_score, 
        name='No warning reference',
        mode="lines", 
        line=dict(color="black", dash="dash")
        )
    )
fig.update_layout(
    xaxis_title='Lead day',
    yaxis_title='Mean FIRM score',
    width=600,
    height=400,
    margin=dict(l=0,r=10,b=50,t=10),
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)


In [28]:
fig.write_image("../figures/firm_line.png", scale=5)